In [1]:
import pandas as pd

# 311 data on dog bites

In [2]:
# load mar 1 2019 - aug 31 2023
df1 = pd.read_csv('Cardona-Maguigad_WBEZ_FOIA REF# F055834-090623_SF RECORDS_MARCH 1, 2019 THRU AUGUST 31, 2023_FINAL.csv', skiprows=1)

In [31]:
# load jan 1 2019 - feb 28 2019
df2 = pd.read_csv('Cardona-Maguigad_WBEZ_FOIA REF# F055834-090623_SF RECORDS_JANUARY 1, 2019 THRU FEBRUARY 28, 2019_FINAL.csv', skiprows=1)

In [33]:
# combine
df = pd.concat([df1, df2])

In [41]:
# check
print('length of first two dfs:', len(df1) + len(df2))
print('length of concatenated df:', len(df))

length of first two dfs: 10414
length of concatenated df: 10414


In [43]:
df.head(2)

,Department,Service Request: Type Name,Service Request Number,Date/Time Opened,Service Request Status,Service Request: Description,Flex Answers Concatenated,Street Block,Zip Code,Flex Question: Flex Question Name,Answer,Community Area Name,Community Area,Ward,Date/Time Closed
0,Animal Care and Control,Animal Bite Investigation,SR23-01484967,8/31/2023 6:35 PM,Completed,Victim was walking dog at which time three dog...,Yes-No-Human-Male-28-puncture to left forearm-...,8500 S KILBOURN AVE,60652.0,Activity Outcome,Left Voicemail / Phone Interview,ASHBURN,70.0,18.0,9/2/2023 7:55 PM
1,Animal Care and Control,Animal Bite Investigation,SR23-01484843,8/31/2023 5:58 PM,Completed,event#09692- Offending dog was off the leash a...,"Yes-No-Dog-Aug 31, 2023-4:15:00 PM-Sidewalk & ...",4000 W MELROSE ST,60641.0,Activity Outcome,Left Voicemail / Phone Interview,IRVING PARK,16.0,30.0,9/2/2023 7:48 PM


In [ ]:
# export combined df
df.to_csv('processed/combined_311.csv')

In [67]:
# de-dupe
df.drop_duplicates(subset=['Service Request Number'], keep='first', inplace=True)

In [68]:
# export combined df
df.to_csv('processed/combined_311_deduped.csv')

## how many total complaints and by year?

In [69]:
# convert date time opened
df['Date/Time Opened'] = pd.to_datetime(df['Date/Time Opened'])
df['Year'] = df['Date/Time Opened'].dt.year
df['Month Year'] = df['Date/Time Opened'].dt.strftime('%m/%Y')

In [70]:
# total complaints
len(df)

5800

In [71]:
pd.pivot_table(df,
              values = 'Service Request Number',
              index = 'Year',
              aggfunc = 'count')

,Service Request Number
Year,
2019,1401
2020,1074
2021,1124
2022,1299
2023,902


In [72]:
pd.pivot_table(df,
              values = 'Service Request Number',
              index = 'Month Year',
              aggfunc = 'count')

,Service Request Number
Month Year,
01/2019,89
01/2020,68
01/2021,46
01/2022,58
01/2023,70
02/2019,65
02/2020,74
02/2021,51
02/2022,63


## find out what community areas they are in

In [73]:
len(df[df['Community Area Name'].isnull()])

1516

In [74]:
# use numbered community areas b/c there are only 47 missing
len(df[df['Community Area'].isnull()])

26

In [75]:
# group 2019-2023 data by community area
g = df.groupby('Community Area').size().reset_index(name='count').sort_values('count', ascending=False)
g['pct'] = g['count']/g['count'].sum()

In [76]:
g['Community Area'] = g['Community Area'].astype(int)
g = pd.merge(g, ca[['GEOID', 'GEOG', 'TOT_POP']], left_on='Community Area', right_on='GEOID')

In [77]:
g

,Community Area,count,pct,GEOID,GEOG,TOT_POP
0,31,255,0.044163,31,Lower West Side,33751
1,25,245,0.042432,25,Austin,96557
2,23,181,0.031347,23,Humboldt Park,54165
3,24,179,0.031001,24,West Town,87781
4,22,173,0.029962,22,Logan Square,71665
...,...,...,...,...,...,...
72,37,13,0.002251,37,Fuller Park,2567
73,76,11,0.001905,76,O'Hare,13418
74,34,8,0.001386,34,Armour Square,13890
75,47,8,0.001386,47,Burnside,2527


In [79]:
# export as csv to upload to google sheets
g.to_csv('processed/311_by_community.csv')

# Dog friendly area data

In [80]:
# load geocoded results
dfa = pd.read_csv('flourish/Dog Friendly Areas Spotchecked for Flourish - Sheet1.csv')

In [81]:
dfa.head()

,Dog friendly area name,Address,City,State,Postal Code,Latitude,Longitude,Accuracy Score
0,Anderson (Fred) Dog Friendly Areas,1611 S. Wabash Avenue,Chicago,IL,60601,41.859783,-87.625161,0.98
1,Bartelme (Mary) Dog Friendly Area,115 S. Sangamon,Chicago,IL,60607,41.880252,-87.650668,1.00
2,Belmont Harbor Dog Friendly Area,Belmont & Lake Shore Drive,Chicago,IL,60614,41.940411,-87.638433,0.69
3,Bryn Mawr Dog Friendly Area - TEMPORARY,Bryn Mawr & Lake Shore Drive,Chicago,IL,60614,41.984343,-87.651897,0.80
4,Calumet Dog Friendly Area,9801 S. Avenue G,Chicago,IL,60617,41.716864,-87.531005,1.00


# City Clerk Data

In [8]:
# copied the processed/city_clerk_data_combined_2018_2023.csv data from dog-breeds (cleaned in dog-breeds)
clerk = pd.read_csv('city_clerk_data_combined_2018_2023.csv')

In [9]:
clerk.head()

,Id,Status,Created Date,Created By,Modified Date,Modified By,Vacc Number,Vacc City,Vacc County,Vacc State,Vacc Expiration,Breed,Color,Sterilized,Gender,Owner Id,Zip Code
0,1187329,ACTIVE,11/12/2021,27516,11/9/2022,15578,21-223409,CHICAGO,COOK,IL,11/8/2023,DACHSHUND MINIATURE LONG HAIRED,TRI,Y,F,310050,60638.0
1,1187328,ACTIVE,11/12/2021,24019,11/12/2021,24019,0356129,CHICAGO,COOK,IL,7/12/2023,CORGI MIX,BUFF/TAN,Y,M,1237065,60616.0
2,1187327,ACTIVE,11/12/2021,24019,11/12/2021,24019,0356129,CHICAGO,COOK,IL,7/12/2023,CORGI MIX,BUFF/TAN,Y,M,1237065,60616.0
3,1187326,ACTIVE,11/5/2021,390245,11/5/2021,390245,11-318913,CHICAGO,COOK,IL,11/1/2022,BORDER COLLIE,WHITE/BLACK,Y,M,1533145,60657.0
4,1187325,ACTIVE,11/5/2021,390245,11/5/2021,390245,11-188032,CHICAGO,COOK,IL,8/21/2022,ROTTWEILER,BLACK/TAN,N,M,1585898,60628.0


## find out how many dogs with current vaccines registered there are

### method 1: active registrations

In [30]:
clerk.groupby('Status').size().reset_index()

,Status,0
0,ACTIVE,78587
1,INACTIVE,5804


### method 2: vacciation status

In [29]:
# convert vacc exp to string
clerk['Vacc Expiration'] = clerk['Vacc Expiration'].astype(str)

# split string to component parts 
clerk[['Month', 'Day', 'Year']] = clerk['Vacc Expiration'].str.split('/', expand=True)

# convert component parts to int so I can sort
clerk['Month'] = clerk['Month'].astype(int)
clerk['Day'] = clerk['Day'].astype(int)
clerk['Year'] = clerk['Year'].astype(int)

# filter for just current (as of Aug. 21, 2023 which is the date of thte dataset) vacciations
clerk_current = clerk[(clerk['Year'] >= 2023) & (clerk['Month'] >= 8) & (clerk['Day'] >= 21)].copy()

# inspect filter - looks good
clerk_current.to_csv('clerk_current.csv')

# print
print('# of dogs with current vacciations registered with the city is: ', len(clerk_current))

print('# of total dogs registered in clerk data from 2018-2023 is:', len(clerk))

# of dogs with current vacciations registered with the city is:  3131
# of total dogs registered in clerk data from 2018-2023 is: 84391


In [20]:
clerk.tail()

,Id,Status,Created Date,Created By,Modified Date,Modified By,Vacc Number,Vacc City,Vacc County,Vacc State,Vacc Expiration,Breed,Color,Sterilized,Gender,Owner Id,Zip Code,Month,Day,Year
84386,1226005,ACTIVE,8/20/2023,EZBUY,NaN,NaN,31-091369,CHICAGO,COOK,IL,7/31/2024,HUNGARIAN VIZSLA,RED/RUST,N,M,1452401,60622.0,7,31,2024
84387,1225949,ACTIVE,8/20/2023,EZBUY,NaN,NaN,20233576418,CHICAGO,COOK,IL,8/10/2026,BEAGLE,TAN/WHITE,Y,M,1231463,60657.0,8,10,2026
84388,1226006,ACTIVE,8/20/2023,EZBUY,NaN,NaN,NaN,CHICAGO,COOK,IL,3/14/2024,AUSTRALIAN SHEPHERD DOG,WHITE/BEIGE,Y,M,1750226,60607.0,3,14,2024
84389,1226050,ACTIVE,8/20/2023,EZBUY,NaN,NaN,311314434,CHICAGO,COOK,IL,8/10/2026,BULL TERRIER,BROWN/GOLDEN,Y,F,1702276,60609.0,8,10,2026
84390,1226051,INACTIVE,8/20/2023,EZBUY,8/20/2023,EZBUY,NaN,CHICAGO,COOK,IL,4/20/2024,UNKNOWN,WHITE/BLACK,N,F,1717076,60623.0,4,20,2024
